In [1]:
import scipy.io

file_path = '../EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)

EV_data = data['EV_re_im_split']
data = EV_data
del EV_data
print(data.shape)

(50, 1000, 832)


In [2]:
# Function to create sequences
from sklearn.model_selection import train_test_split
import numpy as np



def create_sequences(data, timesteps_in,timesteps_out):
    X, y = [], []
    for ue in data:
        # print(len(ue))
        # break
        for i in range(len(ue) - timesteps_in - timesteps_out):
            X.append(ue[i:i + timesteps_in])
            y.append(ue[i + timesteps_in: i + timesteps_in+timesteps_out])
    return np.array(X), np.array(y)

timesteps_in = 10

X, y = create_sequences(data, timesteps_in,5)
print(f'X shape: {X.shape}, y shape: {y.shape}')


# Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


del X
del y


X shape: (49250, 10, 832), y shape: (49250, 5, 832)


In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Bidirectional, Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Verify TensorFlow installation
import tensorflow as tf


2024-05-28 12:00:38.752522: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Define a function to create the advanced LSTM model with gradient clipping
def create_model(optimizer='adam', dropout_rate=0.2, lstm_units=256, dense_units=512, use_bidirectional=False):
    model = Sequential()
    model.add(Input(shape=(timesteps_in, 832)))
    if use_bidirectional:
        model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
    else:
        model.add(LSTM(lstm_units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    
    # if use_bidirectional:
    #     model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
    # else:
    #     model.add(LSTM(lstm_units, return_sequences=True))
    # model.add(Dropout(dropout_rate))
    # model.add(BatchNormalization())
    
    if use_bidirectional:
        model.add(Bidirectional(LSTM(lstm_units)))
    else:
        model.add(LSTM(lstm_units))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    
    # model.add(Dense(dense_units, activation='relu'))
    # model.add(Dropout(dropout_rate))
    # model.add(BatchNormalization())
    model.add(Dense(832))
    
    # Use gradient clipping
    # if optimizer == 'adam':
    optimizer = Adam(clipvalue=1.0)
    # elif optimizer == 'rmsprop':
    #     optimizer = RMSprop(clipvalue=1.0)

    model.compile(optimizer=optimizer, loss='mse')
    return model



In [5]:

best_params = {

    'optimizer': 'adam',
    'dropout_rate': 0.2,
    'lstm_units': 512,
    'dense_units': 512,
    'batch_size': 512,
    'epochs': 100,
    'use_bidirectional': True
}

# Create and train the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

model = create_model(
    optimizer=best_params['optimizer'],
    dropout_rate=best_params['dropout_rate'],
    lstm_units=best_params['lstm_units'],
    dense_units=best_params['dense_units'],
    use_bidirectional=best_params['use_bidirectional']
)



2024-05-28 12:00:40.782638: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 12:00:40.793728: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 12:00:40.796035: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
history = model.fit(
    X_train, y_train,
    batch_size=best_params['batch_size'],
    epochs=best_params['epochs'],
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1)


Epoch 1/100


2024-05-28 12:00:48.189605: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-05-28 12:00:49.263078: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: required broadcastable shapes
2024-05-28 12:00:49.263127: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: required broadcastable shapes
	 [[{{function_node __inference_one_step_on_data_5464}}{{node compile_loss/mse/sub}}]]


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/mse/sub defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/rithvik/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/rithvik/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/home/rithvik/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/home/rithvik/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/rithvik/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/rithvik/anaconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/home/rithvik/anaconda3/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/home/rithvik/anaconda3/lib/python3.